In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")
print(v_data_source)

In [0]:
%run "../Includes/Configuration"

In [0]:
%run "../Includes/Common_Functions"

In [0]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType

In [0]:
circuits_schema = StructType([
    StructField('circuitId', IntegerType(),True),
    StructField('circuitRef', StringType(),True),
    StructField('name', StringType(),True),
    StructField('location', StringType(), True),
    StructField('country', StringType(), True),
    StructField('lat', DoubleType(), True),
    StructField('lng', DoubleType(), True),
    StructField('alt', DoubleType(), True),
    StructField('url', StringType(), True)
])

In [0]:
circuits_renamed_df = spark.read.csv('abfss://udayadbcontainerv1@udayadbsav1.dfs.core.windows.net/RAW-V1/circuits.csv',schema=circuits_schema)

In [0]:
# datasource
from pyspark.sql.functions import lit
circuits_renamed_df = circuits_renamed_df.withColumnRenamed("circuitId","circuit_Id")\
                                         .withColumnRenamed("circuitRef","circuit_Ref")\
                                         .withColumnRenamed('lat', 'latitude')\
                                         .withColumnRenamed('lng', 'longitude')\
                                         .withColumnRenamed('alt', 'altitude')\
                                         .withColumn('data_source',lit(v_data_source))

In [0]:
circuits_final_df = add_ingestion_date(circuits_renamed_df)

In [0]:
circuits_final_df.display()

In [0]:
circuits_final_df.write.mode("overwrite").parquet(f"{Processed_path}/circuits")

In [0]:
display(spark.read.parquet(f"{Processed_path}/circuits"))

In [0]:
dbutils.notebook.exit("Sucessed")